In [0]:
! pip install -q pyspark==3.1.2 spark-nlp

WARNING: You are using pip version 21.0.1; however, version 22.3 is available.
You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command.

In [0]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.config('spark.driver.maxResultSize', '18g').getOrCreate()

In [0]:
import json
import pandas as pd
import numpy as np
# Import pyspark
from pyspark.sql import SparkSession
from pyspark.ml import PipelineModel
from pyspark.sql import functions as F

# Import SparkNLP
import sparknlp
from sparknlp.annotator import *
from sparknlp.base import *


In [0]:
stopwords = StopWordsCleaner().getStopWords()

In [0]:
document = DocumentAssembler() \
            .setInputCol("alltext") \
            .setOutputCol("document")

sentenceDetector = SentenceDetector() \
            .setInputCols("document") \
            .setOutputCol("sentence")

token = Tokenizer() \
            .setInputCols("sentence") \
            .setOutputCol("token") \
            .setContextChars(["(", ")", "?", "!", ".", ","])

keywords = YakeKeywordExtraction() \
            .setInputCols("token") \
            .setOutputCol("keywords") \
            .setMinNGrams(1) \
            .setMaxNGrams(3)\
            .setNKeywords(3)\
            .setStopWords(stopwords)

yake_pipeline = Pipeline(stages=[document, sentenceDetector, token, keywords])

empty_df = spark.createDataFrame([['']]).toDF("alltext")

yake_Model = yake_pipeline.fit(empty_df)

In [0]:
from pyspark.sql.functions import udf
from pyspark.sql.types import *

In [0]:
from pyspark.sql.functions import split, explode, concat, concat_ws

In [0]:
def isValid(lst):
    setA = set(lst)
    setB ={"ukraine","russia","russian","ukraina","putin","kyiv","moscow","invasion","vladimir"}
    return len(setA & setB)>0

In [0]:
isValid_f = udf(isValid, BooleanType())

In [0]:
spark.udf.register(name="isValid_f", f=isValid_f)

Out[10]: <function __main__.isValid(lst)>

In [0]:
# extract keywords from sparkdataframe
def keyword_extract_fromdf(df):
    df1 = df.withColumn("alltext",concat(df['title'],df['selftext']))
    result = yake_pipeline.fit(df1).transform(df1)
    result = result.withColumn('unique_keywords', F.array_distinct("keywords.result"))
    result1 =result.withColumn("isValid",isValid_f('unique_keywords'))
    finaldf=result1.select('author','title','selftext','subreddit','id','score','created_utc').filter('isValid')
    return finaldf

In [0]:
def handle(year_range = [2015, 2016, 2017], month_range = range(1, 13)):
    for year in year_range:
        for month in month_range:
            
            month_literal = str(month)
            data_path = "/mnt/lsde/group05/submissions_tree_shaking/" + str(year) + "-" + month_literal + ".parquet"
            print("start handling "+str(year) + "-" + month_literal)
            df =spark.read.load(data_path,format="parquet")
            df_done = keyword_extract_fromdf(df)
            df_done.write.mode("overwrite").format("parquet").save("/mnt/lsde/group05/"+"kwfilter_top3/"+str(year) + "-" + month_literal+".parquet")
            print("finish handling "+str(year) + "-" + month_literal)
            
    
    
    

In [0]:
handle([2014],[2])

start handling 2014-2
finish handling 2014-2

In [0]:
handle([2014],range(3,13))

start handling 2014-3
finish handling 2014-3
start handling 2014-4
finish handling 2014-4
start handling 2014-5
finish handling 2014-5
start handling 2014-6
finish handling 2014-6
start handling 2014-7
finish handling 2014-7
start handling 2014-8
finish handling 2014-8
start handling 2014-9
finish handling 2014-9
start handling 2014-10
finish handling 2014-10
start handling 2014-11
finish handling 2014-11
start handling 2014-12
finish handling 2014-12

In [0]:
handle([2018],range(8,13))

start handling 2018-8
finish handling 2018-8
start handling 2018-9
finish handling 2018-9
start handling 2018-10
finish handling 2018-10
start handling 2018-11
finish handling 2018-11
start handling 2018-12
finish handling 2018-12

In [0]:
handle([2019,2020,2021],range(1,13))

start handling 2019-1
finish handling 2019-1
start handling 2019-2
finish handling 2019-2
start handling 2019-3
finish handling 2019-3
start handling 2019-4
finish handling 2019-4
start handling 2019-5
finish handling 2019-5
start handling 2019-6
finish handling 2019-6
start handling 2019-7
finish handling 2019-7
start handling 2019-8
finish handling 2019-8
start handling 2019-9
finish handling 2019-9
start handling 2019-10
finish handling 2019-10
start handling 2019-11
finish handling 2019-11
start handling 2019-12
finish handling 2019-12
start handling 2020-1
finish handling 2020-1
start handling 2020-2
finish handling 2020-2
start handling 2020-3
finish handling 2020-3
start handling 2020-4
finish handling 2020-4
start handling 2020-5
finish handling 2020-5
start handling 2020-6
finish handling 2020-6
start handling 2020-7
finish handling 2020-7
start handling 2020-8
finish handling 2020-8
start handling 2020-9
finish handling 2020-9
start handling 2020-10
finish handling 2020-10
start handling 2020-11
finish handling 2020-11
start handling 2020-12
finish handling 2020-12
start handling 2021-1
finish handling 2021-1
start handling 2021-2
finish handling 2021-2
start handling 2021-3
finish handling 2021-3
start handling 2021-4
finish handling 2021-4
start handling 2021-5
finish handling 2021-5
start handling 2021-6
finish handling 2021-6
start handling 2021-7
finish handling 2021-7
start handling 2021-8
finish handling 2021-8
start handling 2021-9
finish handling 2021-9
start handling 2021-10
finish handling 2021-10
start handling 2021-11

--------------------------------------------------------------------------- 
 AnalysisException Traceback (most recent call last)
 <command-707447905985706> in <module> 
 ----> 1 handle ( [ 2019 , 2020 , 2021 ] , range ( 1 , 13 ) ) 

 <command-707447905984426> in handle (year_range, month_range) 
 7 print ( "start handling " + str ( year ) + "-" + month_literal ) 
 8 df = spark . read . load ( data_path , format = "parquet" ) 
 ----> 9 df_done = keyword_extract_fromdf ( df ) 
 10 df_done . write . mode ( "overwrite" ) . format ( "parquet" ) . save ( "/mnt/lsde/group05/" + "kwfilter_top3/" + str ( year ) + "-" + month_literal + ".parquet" ) 
 11 print ( "finish handling " + str ( year ) + "-" + month_literal ) 

 <command-707447905984424> in keyword_extract_fromdf (df) 
 1 # extract keywords from sparkdataframe 
 2 def keyword_extract_fromdf ( df ) : 
 ----> 3 df1 = df . withColumn ( "alltext" , concat ( df [ 'title' ] , df [ 'selftext' ] ) ) 
 4 result = yake_pipeline . fit ( df1 ) . transform ( df1 ) 
 5 result = result . withColumn ( 'unique_keywords' , F . array_distinct ( "keywords.result" ) ) 

 /databricks/spark/python/pyspark/sql/dataframe.py in __getitem__ (self, item) 
 1775 """
 1776 if isinstance ( item , str ) : 
 -> 1777 jc = self . _jdf . apply ( item ) 
 1778 return Column ( jc ) 
 1779 elif isinstance ( item , Column ) : 

 /databricks/spark/python/lib/py4j-0.10.9.1-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1302 
 1303 answer = self . gateway_client . send_command ( command ) 
 -> 1304 return_value = get_return_value(
 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 121 # Hide where the exception came from that shows a non-Pythonic 
 122 # JVM exception message. 
 --> 123 raise converted from None 
 124 else : 
 125 raise 

 AnalysisException : Cannot resolve column name "selftext" among (author, title, subreddit, subreddit_id, id, score, created_utc)

In [0]:
handle([2021],[11,12])

start handling 2021-11
finish handling 2021-11
start handling 2021-12
finish handling 2021-12

In [0]:
handle([2022],range(1,9))

start handling 2022-1
finish handling 2022-1
start handling 2022-2
finish handling 2022-2
start handling 2022-3
finish handling 2022-3
start handling 2022-4
finish handling 2022-4
start handling 2022-5
finish handling 2022-5
start handling 2022-6
finish handling 2022-6
start handling 2022-7
finish handling 2022-7
start handling 2022-8
finish handling 2022-8

In [0]:
handle([2015,2016,2017,2018,2019,2020,2021],range(1,13))

java.lang.RuntimeException: abort: DriverClient destroyed
	at com.databricks.backend.daemon.driver.DriverClient.$anonfun$poll$3(DriverClient.scala:591)
	at scala.concurrent.Future.$anonfun$flatMap$1(Future.scala:307)
	at scala.concurrent.impl.Promise.$anonfun$transformWith$1(Promise.scala:41)
	at scala.concurrent.impl.CallbackRunnable.run(Promise.scala:64)
	at com.databricks.threading.NamedExecutor$$anon$2.$anonfun$run$1(NamedExecutor.scala:360)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at com.databricks.logging.UsageLogging.$anonfun$withAttributionContext$1(UsageLogging.scala:398)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.AttributionContext$.withValue(AttributionContext.scala:142)
	at com.databricks.logging.UsageLogging.withAttributionContext(UsageLogging.scala:396)
	at com.databricks.logging.UsageLogging.withAttributionContext$(UsageLogging.scala:393)
	at com.databricks.threading.NamedExecutor.withAttributionContext(NamedExecutor.scala:287)
	at com.databricks.threading.NamedExecutor$$anon$2.run(NamedExecutor.scala:359)
	at java.base/java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1128)
	at java.base/java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:628)
	at java.base/java.lang.Thread.run(Thread.java:834)

In [0]:
path="dbfs:/mnt/lsde/group05/submissions_tree_shaking/2014-2.parquet"